# SVM Implementation

I did separate preprocessing from the preprocessing.ipynb because SVM was maximizing accuracy by just labeling all data as 0.

In [ ]:
import pandas as pd
import itertools
import numpy as np
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

df = pd.read_csv("loan_approval_dataset.csv")
df.drop(columns=["Id"], inplace=True)
df["Risk_Flag"] = df["Risk_Flag"].apply(lambda x: 1 if x == 1 else -1)

X = df.drop(columns=["Risk_Flag"])
y = df["Risk_Flag"]

X = pd.get_dummies(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=9, stratify=y
)

### Simple Test to see that SVM training is working.

In [4]:
linear_svm = SVC(kernel="linear", class_weight="balanced", random_state=9, max_iter=1000, verbose=True)

linear_svm.fit(X_train, y_train)

y_pred_linear = linear_svm.predict(X_test)

print("Linear SVM Performance:")
print("Accuracy: {:.4f}".format(accuracy_score(y_test, y_pred_linear)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_linear))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_linear))

[LibSVM]

c:\Users\codyb\.vscode\Py\COMP7970-Homework\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Linear SVM Performance:
Accuracy: 0.4360
Confusion Matrix:
[[18266 25935]
 [ 2491  3708]]

Classification Report:
              precision    recall  f1-score   support

          -1       0.88      0.41      0.56     44201
           1       0.13      0.60      0.21      6199

    accuracy                           0.44     50400
   macro avg       0.50      0.51      0.38     50400
weighted avg       0.79      0.44      0.52     50400



Before I got to these behemoth parameter grids, I was not being organized and ran a bunch of training loops for a bunch of hyperparams. I ended up deleting the results and just going with one big iteration that I am going to take as the final hyperparameter testing results

I have tested different max_iters values and it did not seem to effect the results.

In [16]:
customClassWeight1 = {1: 3, -1: 1}
customClassWeight2 = {1: 2, -1: 1}
class_weight_options = ["balanced", customClassWeight1, customClassWeight2]

max_iters = [250]
linear_params = {
    "kernel": ["linear"],
    "C": [0.1, 1, 10, 100],
    "class_weight": class_weight_options
}
rbf_params = {
    "kernel": ["rbf"],
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "gamma": [0.001, 0.01, 0.1, 1, 5, 10, 100],
    "class_weight": class_weight_options
}
poly_params = {
    "kernel": ["poly"],
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "gamma": [0.001, 0.01, 0.1, 1, 5, 10, 100],
    "degree": [2, 3, 4],
    "class_weight": class_weight_options
}

In [17]:
for max_iter_value in max_iters:
    print(f"\n{'='*30}\nStarting tests for max_iter = {max_iter_value}\n{'='*30}\n")

    # Linear
    results_linear = []
    linear_keys = list(linear_params.keys())
    linear_values = list(linear_params.values())
    for combo in itertools.product(*linear_values):
        params = dict(zip(linear_keys, combo))
        params["max_iter"] = max_iter_value
        
        # Train
        model = SVC(random_state=9, verbose=False, **params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        report_text = classification_report(y_test, y_pred, zero_division=0)
        
        # Results
        results_linear.append({
            "kernel": params["kernel"],
            "C": params["C"],
            "gamma": None,
            "degree": None,
            "class_weight": params["class_weight"],
            "max_iter": max_iter_value,
            "accuracy": acc,
            "classification_report": report_text
        })
        
        # Print results
        print(f"[Linear] max_iter: {max_iter_value}, C: {params['C']}, class_weight: {params['class_weight']} -> accuracy: {acc:.4f}")
        print(report_text)
    
    # Write results
    df_linear = pd.DataFrame(results_linear)
    linear_csv_filename = f"linear_report.csv"
    df_linear.to_csv(linear_csv_filename, index=False)
    print(f"Saved linear report to {linear_csv_filename}\n")
    
    # RBF
    results_rbf = []
    rbf_keys = list(rbf_params.keys())
    rbf_values = list(rbf_params.values())
    for combo in itertools.product(*rbf_values):
        params = dict(zip(rbf_keys, combo))
        params["max_iter"] = max_iter_value

        # Train
        model = SVC(random_state=9, verbose=False, **params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        report_text = classification_report(y_test, y_pred, zero_division=0)

        # Results
        results_rbf.append({
            "kernel": params["kernel"],
            "C": params["C"],
            "gamma": params["gamma"],
            "degree": None,  # Not applicable for RBF
            "class_weight": params["class_weight"],
            "max_iter": max_iter_value,
            "accuracy": acc,
            "classification_report": report_text
        })

        # Print results
        print(f"[RBF] max_iter: {max_iter_value}, C: {params['C']}, gamma: {params['gamma']}, class_weight: {params['class_weight']} -> accuracy: {acc:.4f}")
        print(report_text)
    
    # Write results
    df_rbf = pd.DataFrame(results_rbf)
    rbf_csv_filename = f"rbf_report.csv"
    df_rbf.to_csv(rbf_csv_filename, index=False)
    print(f"Saved RBF report to {rbf_csv_filename}\n")
    
    # Polynomial
    results_poly = []
    poly_keys = list(poly_params.keys())
    poly_values = list(poly_params.values())
    for combo in itertools.product(*poly_values):
        params = dict(zip(poly_keys, combo))
        params["max_iter"] = max_iter_value
        
        # Train
        model = SVC(random_state=9, verbose=False, **params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        report_text = classification_report(y_test, y_pred, zero_division=0)
        
        # Results
        results_poly.append({
            "kernel": params["kernel"],
            "C": params["C"],
            "gamma": params["gamma"],
            "degree": params["degree"],
            "class_weight": params["class_weight"],
            "max_iter": max_iter_value,
            "accuracy": acc,
            "classification_report": report_text
        })
        
        # Print results
        print(f"[Poly] max_iter: {max_iter_value}, C: {params['C']}, gamma: {params['gamma']}, degree: {params['degree']}, class_weight: {params['class_weight']} -> accuracy: {acc:.4f}")
        print(report_text)
    
    # Write results
    df_poly = pd.DataFrame(results_poly)
    poly_csv_filename = f"poly_{max_iter_value}_report.csv"
    df_poly.to_csv(poly_csv_filename, index=False)
    print(f"Saved poly report to {poly_csv_filename}\n")


Starting tests for max_iter = 250



c:\Users\codyb\.vscode\Py\COMP7970-Homework\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[Linear] max_iter: 250, C: 0.1, class_weight: balanced -> accuracy: 0.2865
              precision    recall  f1-score   support

          -1       0.89      0.21      0.34     44201
           1       0.13      0.82      0.22      6199

    accuracy                           0.29     50400
   macro avg       0.51      0.51      0.28     50400
weighted avg       0.80      0.29      0.33     50400



c:\Users\codyb\.vscode\Py\COMP7970-Homework\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[Linear] max_iter: 250, C: 0.1, class_weight: {1: 3, -1: 1} -> accuracy: 0.2464
              precision    recall  f1-score   support

          -1       0.89      0.16      0.27     44201
           1       0.13      0.87      0.22      6199

    accuracy                           0.25     50400
   macro avg       0.51      0.51      0.25     50400
weighted avg       0.80      0.25      0.26     50400



...

[Poly] max_iter: 250, C: 100, gamma: 100, degree: 4, class_weight: balanced -> accuracy: 0.8770
              precision    recall  f1-score   support

          -1       0.88      1.00      0.93     44201
           1       0.43      0.00      0.00      6199

    accuracy                           0.88     50400
   macro avg       0.65      0.50      0.47     50400
weighted avg       0.82      0.88      0.82     50400



c:\Users\codyb\.vscode\Py\COMP7970-Homework\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[Poly] max_iter: 250, C: 100, gamma: 100, degree: 4, class_weight: {1: 3, -1: 1} -> accuracy: 0.8770
              precision    recall  f1-score   support

          -1       0.88      1.00      0.93     44201
           1       0.43      0.00      0.00      6199

    accuracy                           0.88     50400
   macro avg       0.65      0.50      0.47     50400
weighted avg       0.82      0.88      0.82     50400



c:\Users\codyb\.vscode\Py\COMP7970-Homework\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=250).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[Poly] max_iter: 250, C: 100, gamma: 100, degree: 4, class_weight: {1: 2, -1: 1} -> accuracy: 0.8770
              precision    recall  f1-score   support

          -1       0.88      1.00      0.93     44201
           1       0.43      0.00      0.00      6199

    accuracy                           0.88     50400
   macro avg       0.65      0.50      0.47     50400
weighted avg       0.82      0.88      0.82     50400

Saved poly report to poly_250_report.csv



### Parser Logic

In [ ]:
def parsePrecisionAndRecall(report, label):
    match = re.search(rf"^\s*{label}\s+([\d.]+)\s+([\d.]+)", report, re.MULTILINE)
    if match:
        precision, recall = float(match.group(1)), float(match.group(2))
        return precision, recall
    return 0.0, 0.0

def filterResults(filename, output_filename):
    df = pd.read_csv(filename)
    keep_rows = []

    for _, row in df.iterrows():
        acc = float(row["accuracy"])
        if acc < 0.13 or acc > 0.87:
            continue

        report = row["classification_report"]

        prec_neg, rec_neg = parsePrecisionAndRecall(report, "-1")
        prec_pos, rec_pos = parsePrecisionAndRecall(report, "1")

        if (prec_neg < 0.10 or rec_neg < 0.10 or
            prec_pos < 0.10 or rec_pos < 0.10):
            continue

        keep_rows.append(row)

    filtered_df = pd.DataFrame(keep_rows)
    filtered_df.to_csv(output_filename, index=False)
    print(f"Filtered report saved to {output_filename} with {len(filtered_df)} rows")

filterResults("linear_report.csv", "filtered_linear_report.csv")
filterResults("rbf_report.csv", "filtered_rbf_report.csv")
filterResults("poly_250_report.csv", "filtered_poly_250_report.csv")